<a href="https://colab.research.google.com/github/matardy/ML-DataMining-Homeworks/blob/master/notebooks/Improved_CNN_Model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Necessary Libraries

In [1]:

import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np
import itertools
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential
from tensorflow.keras import callbacks


## Load and Preprocess the Dataset

In [2]:

(train_dataset, test_dataset), dataset_info = tfds.load(
    name='caltech101',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True,
    data_dir='data',
)

num_classes = dataset_info.features['label'].num_classes

def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

train_dataset = train_dataset.map(preprocess_image).shuffle(1000).batch(32)
test_dataset = test_dataset.map(preprocess_image).batch(32)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3060 [00:00<?, ? examples/s]

Shuffling data/caltech101/3.0.1.incompleteYIVRNO/caltech101-train.tfrecord*...:   0%|          | 0/3060 [00:00…

Generating test examples...:   0%|          | 0/6084 [00:00<?, ? examples/s]

Shuffling data/caltech101/3.0.1.incompleteYIVRNO/caltech101-test.tfrecord*...:   0%|          | 0/6084 [00:00<…

Dataset caltech101 downloaded and prepared to data/caltech101/3.0.1. Subsequent calls will reuse this data.


## Define the Model

In [3]:

# Load the MobileNetV2 model but exclude the classification layers
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)

# Freeze the base_model
base_model.trainable = False

# Create new model on top
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)

# Add data augmentation layer
data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2),
])

x = data_augmentation(x)

# Add pooling layer or flatten layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)

# Add final dense layer
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = layers.Dense(num_classes, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)


9406464/9406464 [==============================] - 0s 0us/step


## Compile and Train the Model

In [4]:

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)

history = model.fit(train_dataset, epochs=50, validation_data=test_dataset, callbacks=[early_stopping])


Epoch 1/50
77/77 [==============================] - 19s 84ms/step - loss: 3.4728 - accuracy: 0.2802 - val_loss: 1.7588 - val_accuracy: 0.6748
Epoch 2/50
77/77 [==============================] - 6s 64ms/step - loss: 1.0475 - accuracy: 0.7798 - val_loss: 0.8151 - val_accuracy: 0.8415
Epoch 3/50
77/77 [==============================] - 5s 59ms/step - loss: 0.4854 - accuracy: 0.8971 - val_loss: 0.6306 - val_accuracy: 0.8529
Epoch 4/50
77/77 [==============================] - 5s 56ms/step - loss: 0.3199 - accuracy: 0.9334 - val_loss: 0.5595 - val_accuracy: 0.8529
Epoch 5/50
77/77 [==============================] - 5s 54ms/step - loss: 0.2120 - accuracy: 0.9518 - val_loss: 0.5623 - val_accuracy: 0.8644
Epoch 6/50
77/77 [==============================] - 6s 73ms/step - loss: 0.1545 - accuracy: 0.9702 - val_loss: 0.5071 - val_accuracy: 0.8775
Epoch 7/50
77/77 [==============================] - 5s 56ms/step - loss: 0.1109 - accuracy: 0.9783 - val_loss: 0.4732 - val_accuracy: 0.8775
Epoch 8/50
7

## Evaluate the Model

In [5]:

loss, accuracy = model.evaluate(test_dataset)
print(f"Accuracy on the test set: {accuracy * 100:.2f}%")


20/20 [==============================] - 1s 43ms/step - loss: 0.4244 - accuracy: 0.8791
Accuracy on the test set: 87.91%
